# 1. roboflow 서버의 모델을 통한 예측

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()  # .env 파일 읽기

rf_api_key = os.getenv("RF_API_KEY")

# print(rf_api_key)

In [ ]:
from inference_sdk import InferenceHTTPClient
CLIENT = InferenceHTTPClient(
api_url="https://detect.roboflow.com",
api_key= rf_api_key
)
filename = './images/number2.jpg'
result = CLIENT.infer(filename, model_id="numbers-doqnw/3")
print(result, "\n")

from pprint import pprint
# pprint(result['predictions'])
pprint([result['predictions'][i]['class'] for i in range(len(result['predictions']))])

{'inference_id': '53ee00c2-feff-4cc4-a7b1-8527bfee10ac', 'time': 0.035564307000072404, 'image': {'width': 800, 'height': 453}, 'predictions': [{'x': 140.0, 'y': 159.5, 'width': 64.0, 'height': 67.0, 'confidence': 0.9916525483131409, 'class': '1', 'class_id': 1, 'detection_id': '304910a2-afc3-4f56-8f71-0a84b3c1b392'}, {'x': 268.5, 'y': 307.5, 'width': 69.0, 'height': 59.0, 'confidence': 0.9910754561424255, 'class': '7', 'class_id': 7, 'detection_id': 'aa77b4a9-bfa0-4d83-b978-055edadc49a4'}, {'x': 544.5, 'y': 163.0, 'width': 77.0, 'height': 76.0, 'confidence': 0.978850245475769, 'class': '4', 'class_id': 4, 'detection_id': 'dcac00ec-bf26-4cd7-8b31-8c3de5cb40a5'}, {'x': 110.5, 'y': 304.5, 'width': 63.0, 'height': 69.0, 'confidence': 0.977420449256897, 'class': '6', 'class_id': 6, 'detection_id': 'da326ff3-115d-4055-ba60-7c49b8f0146b'}, {'x': 516.5, 'y': 301.0, 'width': 53.0, 'height': 74.0, 'confidence': 0.9474438428878784, 'class': '9', 'class_id': 9, 'detection_id': '8ed7242c-862a-412f-

## 바운딩 박스 그리기

In [ ]:
import cv2
img = cv2.imread(filename)
for pred in result["predictions"]:
    x, y = pred['x'], pred['y']
    width, height = pred['width'], pred['height']
    conf = pred['confidence']
    cls = pred['class']
    x1, y1 = int(x-width/2), int(y-height/2)
    x2, y2 = int(x+width/2), int(y+height/2)
    cv2.rectangle(img, (x1,y1), (x2, y2), (0,0,255), 2)
    cv2.putText(img, f'{cls} {conf:.4f}', (x1,y1), cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255))

cv2.imshow('image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 2. 모델 로컬pc에 다운로드 하여 예측 

In [7]:
from roboflow import Roboflow
rf = Roboflow(api_key=rf_api_key)
project = rf.workspace().project("numbers-doqnw")
model = project.version(3).model

filename = './images/number2.jpg'
result = model.predict(filename).json()

loading Roboflow workspace...
loading Roboflow project...


## 바운딩 박스 그리기

In [11]:
import cv2

# 이미지 파일 읽기
img = cv2.imread(filename)

# 이미지의 세로(height), 가로(width) 추출
img_height, img_width = img.shape[:2]

# YOLO 모델이 640x640 기준으로 예측했을 경우 스케일 비율 계산
x_ratio, y_ratio = 640 / img_width, 640 / img_height
print(img_width, img_height, x_ratio, y_ratio)

# 예측된 객체(predictions) 반복 처리
for pred in result["predictions"]:

    # 중심 좌표(x, y), 박스 크기(width, height)
    x, y = pred['x'], pred['y']
    w, h = pred['width'], pred['height']

    # 클래스 이름(class), 신뢰도(confidence)
    cls, conf = pred['class'], pred['confidence']

    # YOLO 형식(center x, center y, width, height)을
    # 좌측상단(x1,y1), 우측하단(x2,y2) 좌표로 변환
    x1 = int(x - w/2)
    y1 = int(y - h/2)
    x2 = int(x + w/2)
    y2 = int(y + h/2)

    # 바운딩박스 그리기
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)

    # 클래스명 + 신뢰도 표시
    cv2.putText(
        img, 
        f'{cls} {conf:.4f}',    # ex) digit 0.9876
        (x1, y1),               # 텍스트 위치 = 박스 좌상단
        cv2.FONT_HERSHEY_PLAIN, # 폰트
        1,                      # 폰트 크기
        (0, 0, 255)             # 빨간색
    )

# 이미지 출력 (창 닫기 전까지 대기)
cv2.imshow('image', img)
cv2.waitKey(0)

# 모든 OpenCV 창 닫기
cv2.destroyAllWindows()


416 416 1.5384615384615385 1.5384615384615385


In [9]:
from roboflow import Roboflow
rf = Roboflow(api_key=rf_api_key)
project = rf.workspace().project("numbers-doqnw")
model = project.version(3).model

filename = './images/number1.jpg'
result = model.predict(filename).json()

loading Roboflow workspace...
loading Roboflow project...


In [14]:
import cv2

img = cv2.imread(filename)
img_height, img_width = img.shape[:2]
x_ratio, y_ratio = 640/img_width, 640/img_height
print(img_width, img_height, x_ratio, y_ratio)
for pred in result["predictions"]:
    # 중심 좌표(x, y), 박스 크기(width, height)
    x, y = pred['x'], pred['y']
    width, height = pred['width'], pred['height']

    # 클래스 이름(class), 신뢰도(confidence)
    conf, cls = pred['confidence'], pred['class']

    # YOLO 형식(center x, center y, width, height)을
    # 좌측상단(x1,y1), 우측하단(x2,y2) 좌표로 변환
    x1, y1 = int(x-width/2), int(y-height/2)
    x2, y2 = int(x+width/2), int(y+height/2)
    x1, x2 = int(x1*x_ratio), int(x2*x_ratio)
    y1, y2 = int(y1*y_ratio), int(y2*y_ratio)
    
     # 바운딩박스 그리기
    cv2.rectangle(img, (x1,y1), (x2, y2), (0,0,255), 2)

    # 클래스명 + 신뢰도 표시
    cv2.putText(img, 
                f'{cls} {conf:.4f}',     # ex) digit 0.9876
                (x1,y1),                 # 텍스트 위치 = 박스 좌상단
                cv2.FONT_HERSHEY_PLAIN,  # 폰트
                1,                       # 폰트 크기
                (0,0,255))               # 빨간색 (BGR)
cv2.imshow('image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


416 416 1.5384615384615385 1.5384615384615385
